In [1]:
import pandas as pd; import numpy as np;#import researchpy as rp
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
# From the  sklearn.model_selection importing the necessary modules
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
# From the  sklearn.linear_model importing the necessary module
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold
# from sklearn.tree importing the necessary module
from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics importing the necessary modules
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score,roc_curve, auc
#Importing the visualization tools
#from IPython.display import Image  
#import graphviz

# Import XGBRegressor
from xgboost import XGBClassifier

# Preprocessing tools
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.tree import export_graphviz, plot_tree
# from sklearn.neighbors importing the necessary module
from sklearn.neighbors import KNeighborsClassifier 
# from.preprocessing importing the necessary modules
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
#from sklearn.ensemble importing the necessary modules
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier, VotingClassifier

In [2]:
# importing the data and getting the info
url="E:\\machine learning\\project_ml\\creditcard.txt"
credit= pd.read_csv(url)
credit.head()


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [3]:
print('='*50)
credit.info()
print('='*50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          30000 non-null  int64
 1   LIMIT_BAL                   30000 non-null  int64
 2   SEX                         30000 non-null  int64
 3   EDUCATION                   30000 non-null  int64
 4   MARRIAGE                    30000 non-null  int64
 5   AGE                         30000 non-null  int64
 6   PAY_0                       30000 non-null  int64
 7   PAY_2                       30000 non-null  int64
 8   PAY_3                       30000 non-null  int64
 9   PAY_4                       30000 non-null  int64
 10  PAY_5                       30000 non-null  int64
 11  PAY_6                       30000 non-null  int64
 12  BILL_AMT1                   30000 non-null  int64
 13  BILL_AMT2                   30000 non-null  int64
 14  BILL_A

In [4]:
# check missingness
credit.isnull().sum()

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64

In [5]:


# Splitting the data into train and test sets

X = credit.iloc[:, :-1]
y = credit.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2,
                                                   random_state=101)
                                                 

In [12]:
xgb_clf = XGBClassifier(booster='gbtree', 
                    objective='binary:logistic',
                    random_state=2,
                    use_label_encoder=False,
                    eval_metric='logloss')
    
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
    

params={'n_estimators':[2, 25, 50, 75, 100, 200],
        'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
        'max_depth':[1, 2, 3, 4, None],
        'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
        'min_child_weight':[1, 2, 3, 4, 5],
        'colsample_bynode': [0.5, 0.6, 0.7, 0.8, 0.9, 1], 
        'colsample_bylevel':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 
        'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1],
        'gamma':[0, 0.01, 0.05, 0.1, 0.5, 1, 2, 3],
        'alpha': [1, 10, 50, 100],
        'lambda': [1, 10, 50, 100]}


rs_grid = RandomizedSearchCV(estimator=xgb_clf, 
                            param_distributions=params,
                            cv=kfold, 
                            n_iter=500, 
                            n_jobs=-1, 
                            random_state=2)

# Fit grid_reg on X_train and y_train
# -----------------------------------
rs_grid.fit(X, y)
      
# Print best params
for param, val in rs_grid.best_params_.items():
    print("{:18} : {}".format(param, val))

subsample          : 0.8
n_estimators       : 75
min_child_weight   : 2
max_depth          : None
learning_rate      : 0.05
lambda             : 100
gamma              : 0.1
colsample_bytree   : 0.8
colsample_bynode   : 0.9
colsample_bylevel  : 0.7
alpha              : 10


In [15]:
xgb_clf = XGBClassifier(booster='gbtree', 
                      objective='binary:logistic',
                      random_state=101, 
                      use_label_encoder=False,
                      eval_metric = 'logloss',
                    learning_rate= 0.05, max_depth= 4, min_child_weight=3, n_estimators=75, subsample= 0.8,
                         gamma=0.1,
                        colsample_bytree=0.8,
                       colsample_bynode=0.9,
                        colsample_bylevel=0.7,
                             alpha=10)

In [16]:

# Fit the classifier
# ------------------
xgb_clf.fit(X_train, y_train)

# Generate predictions
# ---------------------
pred = xgb_clf.predict(X_test)

# Score the results
# -----------------

score = accuracy_score(y_test, pred)
print("*"*30)
print('The accuracy score test set: {:.3f}'.format(score))
print("*"*30)

cv_scores = cross_val_score(xgb_clf,
                         X, y, 
                         cv=5)

print("*"*50)
print('Accuracy:', np.round(cv_scores, 2))
print("*"*50)

print('The percentage of mean accuracy: {:0.3f}%'.format((cv_scores.mean()*100)))
print("*"*50)


# Print best score
# -----------------
print("Training score: {:.4f}".format(rs_grid.best_score_))



******************************
The accuracy score test set: 0.826
******************************
**************************************************
Accuracy: [0.81 0.81 0.82 0.83 0.82]
**************************************************
The percentage of mean accuracy: 81.997%
**************************************************
Training score: 0.8224
